# fitness-eval-app — SQLite Database Explorer

This notebook connects to the SQLite database created in **Phase 2** and displays
all four tables for inspection and reference.

**Tables:**
| Table | Description |
|-------|-------------|
| `coaches` | Registered coach accounts (bcrypt passwords) |
| `clients` | Clients owned by a coach — strict per-coach isolation |
| `body_measurements` | Timestamped measurement snapshots per client |
| `assessments` | Timestamped assessment result blobs per client |

> **Run order:** Execute cells top-to-bottom. The setup cell creates the DB
> and migrates data from `clients.json` / `coaches.json` if not already done.

## 1 — Setup

In [1]:
import asyncio
import json
import sqlite3
import sys
from pathlib import Path

import pandas as pd

# ── Resolve paths ─────────────────────────────────────────────────────────────
REPO_ROOT  = Path("../").resolve()
BACKEND    = REPO_ROOT / "backend"
DB_PATH    = BACKEND / "data" / "fitness.db"

# Add backend to sys.path so we can import app modules.
if str(BACKEND) not in sys.path:
    sys.path.insert(0, str(BACKEND))

print(f"Repo root : {REPO_ROOT}")
print(f"Backend   : {BACKEND}")
print(f"DB path   : {DB_PATH}")
print(f"DB exists : {DB_PATH.exists()}")

Repo root : /home/alireza/projects/fitness-eval-app
Backend   : /home/alireza/projects/fitness-eval-app/backend
DB path   : /home/alireza/projects/fitness-eval-app/backend/data/fitness.db
DB exists : True


## 2 — Create Tables + Run Migration

Safe to run repeatedly — migration skips if the `coaches` table is already populated.

In [2]:
# Import ORM models to register them with Base.metadata.
import app.db_models  # noqa: F401  (side-effect: registers tables)
from app.database import AsyncSessionLocal, create_tables
from app.migrate_json_to_db import run_migration_if_needed

async def setup_db() -> None:
    """Create tables and seed data from JSON files (idempotent)."""
    await create_tables()
    async with AsyncSessionLocal() as db:
        await run_migration_if_needed(db)

await setup_db()
print(f"DB ready: {DB_PATH}  ({DB_PATH.stat().st_size / 1024:.1f} KB)")

DB ready: /home/alireza/projects/fitness-eval-app/backend/data/fitness.db  (52.0 KB)


## 3 — Helper: read any table into a DataFrame

In [3]:
def read_table(table: str, limit: int | None = None) -> pd.DataFrame:
    """Read a SQLite table into a pandas DataFrame.

    Args:
        table: Table name.
        limit: Optional row cap (None = all rows).

    Returns:
        DataFrame with all columns.
    """
    sql = f"SELECT * FROM {table}"  # noqa: S608 — notebook only, no user input
    if limit:
        sql += f" LIMIT {limit}"
    with sqlite3.connect(DB_PATH) as con:
        return pd.read_sql_query(sql, con)


def table_info(table: str) -> pd.DataFrame:
    """Return PRAGMA table_info for column definitions."""
    with sqlite3.connect(DB_PATH) as con:
        return pd.read_sql_query(f"PRAGMA table_info({table})", con)


print("Helper functions ready.")

Helper functions ready.


## 4 — `coaches` table

In [4]:
coaches = read_table("coaches")

# Don't expose the hash — mask it for display.
coaches_display = coaches.copy()
coaches_display["hashed_password"] = "[bcrypt hash]" 

print(f"Rows: {len(coaches_display)}")
coaches_display

Rows: 1


,id,username,hashed_password,display_name,created_at
0,1,admin,[bcrypt hash],Admin Coach,2026-02-22 03:38:58.117405


In [5]:
# Column definitions
table_info("coaches")

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,1,None,1
1,1,username,VARCHAR(32),1,None,0
2,2,hashed_password,VARCHAR(128),1,None,0
3,3,display_name,VARCHAR(128),1,None,0
4,4,created_at,DATETIME,1,None,0


## 5 — `clients` table

In [6]:
clients = read_table("clients")

# Parse goals JSON for readability.
clients_display = clients.copy()
clients_display["goals"] = clients_display["goals"].apply(
    lambda g: ", ".join(json.loads(g)) if g else ""
)

print(f"Rows: {len(clients_display)}")
clients_display

Rows: 1


,id,coach_id,name,age,gender,goals,notes,height_cm,preferred_activities,equipment_available,saved_at
0,1,1,Ali Zarreh,42,male,"general_fitness, sport_performance",None,173.0,[],[],2026-02-21 01:13:23.417552


In [7]:
# Column definitions
table_info("clients")

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,1,None,1
1,1,coach_id,INTEGER,1,None,0
2,2,name,VARCHAR(256),1,None,0
3,3,age,INTEGER,1,None,0
4,4,gender,VARCHAR(8),1,None,0
5,5,goals,TEXT,1,None,0
6,6,notes,TEXT,0,None,0
7,7,height_cm,FLOAT,0,None,0
8,8,preferred_activities,TEXT,1,None,0
9,9,equipment_available,TEXT,1,None,0


## 6 — `body_measurements` table

In [8]:
measurements = read_table("body_measurements")

print(f"Rows: {len(measurements)}")
measurements

Rows: 1


,id,client_id,measured_at,weight_kg,waist_cm,hip_cm,neck_cm,bmi,body_fat_pct,body_fat_rating,fat_mass_kg,lean_mass_kg
0,1,1,2026-02-21 01:13:23.417552,87.0,90.0,103.0,None,29.1,None,None,None,None


In [9]:
# Column definitions
table_info("body_measurements")

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,1,None,1
1,1,client_id,INTEGER,1,None,0
2,2,measured_at,DATETIME,1,None,0
3,3,weight_kg,FLOAT,0,None,0
4,4,waist_cm,FLOAT,0,None,0
5,5,hip_cm,FLOAT,0,None,0
6,6,neck_cm,FLOAT,0,None,0
7,7,bmi,FLOAT,0,None,0
8,8,body_fat_pct,FLOAT,0,None,0
9,9,body_fat_rating,VARCHAR(32),0,None,0


## 7 — `assessments` table

In [10]:
assessments = read_table("assessments")

# Parse results_json to show metric count and rating summary instead of raw blob.
def _summarise_results(blob: str) -> str:
    try:
        results = json.loads(blob)
        parts = [f"{r['test_name']}: {r['rating']}" for r in results]
        return " | ".join(parts)
    except Exception:
        return blob

assessments_display = assessments.copy()
assessments_display["results_summary"] = assessments_display["results_json"].apply(_summarise_results)
assessments_display = assessments_display.drop(columns=["results_json"])

print(f"Rows: {len(assessments_display)}")
assessments_display

Rows: 7


,id,client_id,assessed_at,results_summary
0,1,1,2026-02-21 03:14:14.119803,Push-up Test: Excellent | Wall Sit Test: Excel...
1,2,1,2026-02-21 13:02:32.442621,Push-up Test: Excellent | Wall Sit Test: Good ...
2,3,1,2026-02-21 13:18:47.185560,Push-up Test: Excellent | Wall Sit Test: Good ...
3,4,1,2026-02-21 19:30:26.285454,Push-up Test: Excellent | Wall Sit Test: Very ...
4,5,1,2026-02-21 19:46:52.887177,Push-up Test: Excellent | Wall Sit Test: Very ...
5,6,1,2026-02-21 20:29:48.314804,Push-up Test: Excellent | Wall Sit Test: Very ...
6,7,1,2026-02-21 20:53:30.473383,Push-up Test: Very Good | Wall Sit Test: Fair ...


In [11]:
# Column definitions
table_info("assessments")

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,1,None,1
1,1,client_id,INTEGER,1,None,0
2,2,assessed_at,DATETIME,1,None,0
3,3,results_json,TEXT,1,None,0


## 8 — Cross-table join: clients with their coach name

In [12]:
sql = """
SELECT
    cl.id            AS client_id,
    co.username      AS coach,
    cl.name          AS client_name,
    cl.age,
    cl.gender,
    cl.height_cm,
    cl.saved_at,
    COUNT(DISTINCT a.id)  AS assessment_count,
    COUNT(DISTINCT bm.id) AS measurement_count
FROM clients cl
JOIN coaches co ON co.id = cl.coach_id
LEFT JOIN assessments a   ON a.client_id  = cl.id
LEFT JOIN body_measurements bm ON bm.client_id = cl.id
GROUP BY cl.id
ORDER BY co.username, cl.saved_at DESC
"""

with sqlite3.connect(DB_PATH) as con:
    joined = pd.read_sql_query(sql, con)

print(f"Rows: {len(joined)}")
joined

Rows: 1


,client_id,coach,client_name,age,gender,height_cm,saved_at,assessment_count,measurement_count
0,1,admin,Ali Zarreh,42,male,173.0,2026-02-21 01:13:23.417552,7,1
